Libraries

In [68]:
import pandas as pd
#import numpy as np
from sklearn import metrics, svm
from sklearn. model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
sms=pd.read_csv('E:\\shirin\\Kaggle\\6-SMS\\spam.csv',usecols=[0,1],header=0,names=['label', 'message'] ,encoding='latin-1')
print(sms.shape)
print(sms.label.value_counts())
sms.head()

(5572, 2)
label
ham     4825
spam     747
Name: count, dtype: int64


,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


Spilit Dataset to Train and Test Dataset

In [69]:
X=sms.message
y=sms.label
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(4179,)
(4179,)
(1393,)
(1393,)


Vectorizing Our Dataset

In [70]:
vect=CountVectorizer()
X_train_dtm=vect.fit_transform(X_train,y_train)
print(X_train_dtm.shape)
X_test_dtm=vect.transform(X_test)
print(X_test_dtm.shape)

(4179, 7441)
(1393, 7441)


Building Model

In [71]:
nb= MultinomialNB()
%time nb.fit(X_train_dtm,y_train)
y_pred_nb=nb.predict(X_test_dtm)
print(f'Accuracy Score:{metrics.accuracy_score(y_test, y_pred_nb)}')
print(f'Confusion Matrix:\n{metrics.confusion_matrix(y_test, y_pred_nb)}')

CPU times: total: 0 ns
Wall time: 4.75 ms
Accuracy Score:0.9820531227566404
Confusion Matrix:
[[1199    3]
 [  22  169]]


In [72]:
logit=LogisticRegression()
%time logit.fit(X_train_dtm,y_train)
y_pred_logit=logit.predict(X_test_dtm)
print(f'Accuracy Score:{metrics.accuracy_score(y_test, y_pred_logit)}')
print(f'Confusion Matrix:\n{metrics.confusion_matrix(y_test, y_pred_logit)}')

CPU times: total: 15.6 ms
Wall time: 39.2 ms
Accuracy Score:0.9748743718592965
Confusion Matrix:
[[1199    3]
 [  32  159]]


In [73]:
svm=svm.SVC()
%time svm.fit(X_train_dtm,y_train)
y_pred_svm=svm.predict(X_test_dtm)
print(f'Accuracy Score:{metrics.accuracy_score(y_test, y_pred_svm)}')
print(f'Confusion Matrix:\n{metrics.confusion_matrix(y_test, y_pred_svm)}')

CPU times: total: 250 ms
Wall time: 1.01 s
Accuracy Score:0.9777458722182341
Confusion Matrix:
[[1202    0]
 [  31  160]]


Null Accuracy

In [62]:
print(y_test.value_counts())
y_test.value_counts().head(1)/y_test.shape

label
ham     1202
spam     191
Name: count, dtype: int64


label
ham    0.862886
Name: count, dtype: float64

Calculate Spamminess of Each Token

In [63]:
X_train_tokens=vect.get_feature_names_out()
print(X_train_tokens.shape)
print(X_train_tokens[0:10])
print(X_train_tokens[-10:])
nb.feature_count_
X_train_ham=nb.feature_count_[0,:]
X_train_spam=nb.feature_count_[1,:]
tokens=pd.DataFrame({'token':X_train_tokens,'ham':X_train_ham,'spam':X_train_spam}).set_index('token')
tokens['spam']=tokens.spam+1
tokens['ham']=tokens.ham+1
tokens['ratio_ham']=tokens.ham/nb.class_count_[0]
tokens['ratio_spam']=tokens.spam/nb.class_count_[1]
tokens['ratio']=tokens.ratio_spam/tokens.ratio_ham
tokens.sort_values('ratio',ascending=False)

(7441,)
['00' '000' '000pes' '008704050406' '0089' '0121' '0125698789' '02' '0207'
 '02072069400']
['ìï' 'û_' 'û_thanks' 'ûªm' 'ûªt' 'ûªve' 'ûï' 'ûïharry' 'ûò' 'ûówell']


,ham,spam,ratio_ham,ratio_spam,ratio
token,,,,,
claim,1.0,88.0,0.000276,0.158273,573.424460
prize,1.0,69.0,0.000276,0.124101,449.616906
uk,1.0,53.0,0.000276,0.095324,345.357914
150p,1.0,47.0,0.000276,0.084532,306.260791
tone,1.0,42.0,0.000276,0.075540,273.679856
...,...,...,...,...,...
she,126.0,1.0,0.034778,0.001799,0.051716
lor,126.0,1.0,0.034778,0.001799,0.051716
he,166.0,1.0,0.045818,0.001799,0.039254
